In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
path='/content/drive/MyDrive/AI_24/' # με ευκόλο τρόπο μπορείτε να αλλάξετε το path που είναι αποθηκευμένα τα αρχεία σας

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository -y ppa:swi-prolog/devel
!sudo apt-get update
!sudo apt-get install swi-prolog
#install pyswip
!pip install 'git+https://github.com/yuce/pyswip@master#egg=pyswip'

In [ ]:
import pandas as pd
from pyswip import Prolog

In [ ]:

import pandas as pd
path = '/content/drive/MyDrive/AI_24/'

data = pd.read_csv(path + "movies_metadata.csv")

data.fillna("UNK", inplace=True)
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,...,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Enter the World of Pandora.,Avatar,7.2,11800,2009.0,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007.0,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,A Plan No One Escapes,Spectre,6.3,4466,2015.0,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,The Legend Ends,The Dark Knight Rises,7.6,9106,2012.0,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"Lost in our world, found in another.",John Carter,6.1,2124,2012.0,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


In [ ]:
def clean_text(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.lstrip('\n')
    text = text.lstrip('"')
    text = text.rstrip('\n')
    text = text.rstrip(';')
    text = text.rstrip('"')
    text = text.replace('""', '"')
    text = text.replace(';"', '')
    text = text.replace('";', '')
    text = text.replace('&amp"', '&')
    text = text.replace('"', '')
    return text

In [ ]:
#create World
prolog = Prolog()

literals = []
movie_score = {}
for row in data.itertuples(index=True, name='Pandas'):
  movie_title = clean_text(getattr(row, 'movie_title'))

  for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ movie_title +"','"+ genre +"')")

  for plotKeyword in getattr(row, 'plot_keywords').split("|"):
    plotKeyword = plotKeyword.replace("'", '')
    literals.append("plot_keyword('"+movie_title+"','"+plotKeyword+"')")

  literals.append("language('"+movie_title+"','"+getattr(row, 'language')+"')")

  for productionCompany in list(eval(getattr(row, 'production_companies'))):
    pcname = productionCompany["name"].replace("'", "")
    literals.append("production_company('"+movie_title+"','"+pcname+"')")

  name = clean_text(getattr(row, 'director_name'))
  literals.append("director('"+movie_title+"','"+name+"')")

  name = clean_text(getattr(row, 'actor_1_name'))
  literals.append("actor('"+movie_title+"','"+name+"')")

  name = clean_text(getattr(row, 'actor_2_name'))
  literals.append("actor('"+movie_title+"','"+name+"')")

  name = clean_text(getattr(row, 'actor_3_name'))
  literals.append(f"actor('"+movie_title+"','"+name+"')")

  if(getattr(row, 'title_year') != "UNK"):
    year = int(getattr(row, 'title_year'))
    decade = str(year - year % 10)
    literals.append("decade('"+movie_title+"','"+decade+"')")

literals.sort()

for literal in literals:
  prolog.assertz(literal)

prolog.query("close_query.")


<generator object Prolog._QueryWrapper.__call__ at 0x7db368ffc890>

In [ ]:

prolog.consult(path + "db.pl")

In [ ]:
#prolog.assertz('(directed_by(X,Y) :- findall(M,director(M,X),Y))')

In [ ]:
q = prolog.query("atleast_one_common_genre('The SpongeBob SquarePants Movie', M)")
s = set()
for sol in q: s.add(sol['M'])
q.close()
print(s)

{'Lonesome Jim', 'Confessions of a Teenage Drama Queen', 'Crocodile Dundee II', 'I Hope They Serve Beer in Hell', 'The Artist', 'Mr. Nice Guy', 'The Change-Up', 'Mrs Henderson Presents', 'Furry Vengeance', 'Joy', 'Freaky Deaky', 'The Sitter', 'The Whole Ten Yards', 'Whatever It Takes', 'The Good Night', 'Dead Like Me: Life After Death', 'Lucky Numbers', 'Edtv', 'Garfield: A Tail of Two Kitties', 'Lethal Weapon 4', 'Stuart Little 2', 'Renaissance', 'The Lost Boys', 'Planet 51', 'Diary of a Mad Black Woman', 'The Oh in Ohio', 'Guess Who', 'Tiny Furniture', 'Take Me Home Tonight', 'Simply Irresistible', 'Whats the Worst That Could Happen?', 'Post Grad', 'Due Date', 'Elf', 'Ice Age', 'My Big Fat Greek Wedding', 'The Guru', 'Road Trip', 'That Awkward Moment', 'Drive Hard', 'The Brothers Grimm', 'Driving Lessons', 'The Road to El Dorado', 'The Upside of Anger', 'Just My Luck', 'Khumba', 'I Can Do Bad All By Myself', 'Grown Ups 2', 'Raising Helen', 'The Jimmy Show', 'The Tale of Despereaux', 

In [ ]:
q = prolog.query("one_common_company('The Wolf of Wall Street', M)")
for sol in q:
  print(sol)
q.close()

{'M': 'Out of the Furnace'}
{'M': 'Public Enemies'}
{'M': 'Red Riding Hood'}
{'M': 'Runner Runner'}
{'M': 'Shutter Island'}
{'M': 'The Aviator'}
{'M': 'The Ides of March'}
{'M': 'The Revenant'}
{'M': 'Bullet to the Head'}
{'M': '10 Cloverfield Lane'}
{'M': '13 Hours: The Secret Soldiers of Benghazi'}
{'M': '200 Cigarettes'}
{'M': 'A Civil Action'}
{'M': 'A Farewell to Arms'}
{'M': 'A Night at the Roxbury'}
{'M': 'A Simple Plan'}
{'M': 'A Thousand Words'}
{'M': 'Abandon'}
{'M': 'Against the Ropes'}
{'M': 'Airplane!'}
{'M': 'Alfie'}
{'M': 'Alive'}
{'M': 'Along Came a Spider'}
{'M': 'Anchorman 2: The Legend Continues'}
{'M': 'Angelas Ashes'}
{'M': 'April Fools Day'}
{'M': 'Bad Grandpa'}
{'M': 'Barbarella'}
{'M': 'Barnyard'}
{'M': 'Beavis and Butt-Head Do America'}
{'M': 'Ben-Hur'}
{'M': 'Beowulf'}
{'M': 'Beverly Hills Cop II'}
{'M': 'Beverly Hills Cop III'}
{'M': 'Beverly Hills Cop'}
{'M': 'Black Rain'}
{'M': 'Bless the Child'}
{'M': 'Boomerang'}
{'M': 'Breakdown'}
{'M': 'Bringing Out the

In [ ]:
q = prolog.query("same_director('Shutter Island', M)")
s = set()
for sol in q: s.add(sol['M'])
q.close()
print(s)

{'Casino', 'Cape Fear', 'Raging Bull', 'Bringing Out the Dead', 'Mean Streets', 'The Departed', 'Gangs of New York', 'Hugo', 'Shine a Light', 'Taxi Driver', 'GoodFellas', 'The Last Waltz', 'The Wolf of Wall Street', 'New York, New York', 'The Last Temptation of Christ', 'The Color of Money', 'The Age of Innocence', 'Kundun', 'The Aviator'}


In [ ]:
q = prolog.query("atleast_one_common_actor('GoodFellas', M)")
s = set()
for sol in q: s.add(sol['M'])
q.close()
print(s)

{'Identity', 'Casino', 'Youth in Revolt', 'Heartbreakers', 'Smokin Aces', 'Love Ranch', 'Raging Bull', 'Medicine Man', 'Cop Land', 'Once Upon a Time in America', 'Narc', 'The Iceman', 'Lethal Weapon 3', 'Home Alone 2: Lost in New York', 'Radio Flyer', 'In the Name of the King: A Dungeon Siege Tale', 'Kill the Messenger', 'Lethal Weapon 4', 'Blow', 'Observe and Report', 'Home Alone'}


In [ ]:
q = prolog.query("same_director('Home Alone', M)")
s = set()
for sol in q: s.add(sol['M'])
q.close()
print(s)

{'I Love You, Beth Cooper', 'Home Alone 2: Lost in New York', 'Stepmom', 'Rent', 'Harry Potter and the Philosophers Stone', 'Mrs. Doubtfire', 'Percy Jackson & the Olympians: The Lightning Thief', 'Harry Potter and the Chamber of Secrets', 'Pixels', 'Bicentennial Man'}


In [ ]:
def simple_recommender(movie):
    s = set()
    for i in range(1, 5):
        q = prolog.query("find_sim_" + str(i) + "('" + movie +"', M)")
        for sol in q:
            m = sol['M']
            #print(m)
            s.add(sol['M'])
        q.close()
    return s


In [ ]:
list(simple_recommender('Home Alone'))[:5]

['Lonesome Jim',
 'Confessions of a Teenage Drama Queen',
 'Crocodile Dundee II',
 'I Hope They Serve Beer in Hell',
 'The Artist']

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import random

rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {i:i + 1 for i in range(1)} 
def train_recommender(ratings, rating_weights, score_weights, number_of_movies = 10):
   

    if number_of_movies > len(ratings):
        number_of_movies = len(ratings)


    if number_of_movies != -1:
        indexes = random.sample(range(len(ratings)), number_of_movies)
        ratings = ratings.iloc[indexes]

    movie_score = {}
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        similar_movies = simple_recommender(movie)

        for similar_movie in similar_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[0]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[0] 
    return movie_score



def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: 
            pred.append(int(movie_score[movie] > 0)) 
            real.append(int(rating > 3))
        else: 
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

In [ ]:
train_ratings = pd.read_csv(path + "train_ratings.csv")
test_ratings = pd.read_csv(path + "test_ratings.csv")

In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

precision: 0.5126457771691525
recall: 0.825
f1: 0.6155556282113942


In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 30)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

precision: 0.5121480758064973
recall: 0.9680555555555557
f1: 0.6698612808182213


In [ ]:
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, -1)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

precision: 0.5106382978723404
recall: 1.0
f1: 0.676056338028169
